# KL By Removed

In [8]:
import numpy as np
from lib.utils import model_results_from_npz
import csv

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 10
EPOCHS = 50
REMOVED_CLASSES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"
EPS = 1e-12

# --------------------
# Prepare CSV
# --------------------
csv_path = "../analytics/CIFAR10/kl/kl_per_removed.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Seed", "Removed_Class", "KL_Forget", "KL_Retain", "KL_Whole"])

    # --------------------
    # Loop over seeds & removed classes
    # --------------------
    for seed in SEEDS:
        for removed_class in REMOVED_CLASSES:

            # Load models
            retrain_path = f"{DIR_RETRAIN}/cifar_resnet_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # --------------------
            # KL on forget class
            # --------------------
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            # Normalize safely
            conf_retrain_forget /= (conf_retrain_forget.sum(axis=1, keepdims=True) + EPS)
            conf_redis_forget   /= (conf_redis_forget.sum(axis=1, keepdims=True) + EPS)
            kl_forget = np.sum(conf_retrain_forget * np.log((conf_retrain_forget + EPS) / (conf_redis_forget + EPS)), axis=1).mean()

            # --------------------
            # KL on retain classes
            # --------------------
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c]   for c in retain_classes])
            conf_retrain_retain /= (conf_retrain_retain.sum(axis=1, keepdims=True) + EPS)
            conf_redis_retain   /= (conf_redis_retain.sum(axis=1, keepdims=True) + EPS)
            kl_retain = np.sum(conf_retrain_retain * np.log((conf_retrain_retain + EPS) / (conf_redis_retain + EPS)), axis=1).mean()

            # --------------------
            # KL on whole dataset
            # --------------------
            conf_retrain_all = np.vstack([model_retrain.confidences[c] for c in range(NUM_CLASSES)])
            conf_redis_all   = np.vstack([model_redis.confidences[c]   for c in range(NUM_CLASSES)])
            conf_retrain_all /= (conf_retrain_all.sum(axis=1, keepdims=True) + EPS)
            conf_redis_all   /= (conf_redis_all.sum(axis=1, keepdims=True) + EPS)
            kl_whole = np.sum(conf_retrain_all * np.log((conf_retrain_all + EPS) / (conf_redis_all + EPS)), axis=1).mean()

            # Write row
            writer.writerow([seed, removed_class, kl_forget, kl_retain, kl_whole])
            print(f"Seed {seed}, Removed Class {removed_class} | KL Forget: {kl_forget:.6f}, KL Retain: {kl_retain:.6f}, KL Whole: {kl_whole:.6f}")

print(f"CSV saved to {csv_path}")


Seed 42, Removed Class 0 | KL Forget: 0.788176, KL Retain: 0.185609, KL Whole: 0.245866
Seed 42, Removed Class 1 | KL Forget: 0.847545, KL Retain: 0.186457, KL Whole: 0.252566
Seed 42, Removed Class 2 | KL Forget: 0.843244, KL Retain: 0.169673, KL Whole: 0.237030
Seed 42, Removed Class 3 | KL Forget: 0.730075, KL Retain: 0.175179, KL Whole: 0.230668
Seed 42, Removed Class 4 | KL Forget: 0.863450, KL Retain: 0.193312, KL Whole: 0.260326
Seed 42, Removed Class 5 | KL Forget: 0.831816, KL Retain: 0.158705, KL Whole: 0.226016
Seed 42, Removed Class 6 | KL Forget: 1.020230, KL Retain: 0.218535, KL Whole: 0.298705
Seed 42, Removed Class 7 | KL Forget: 0.909507, KL Retain: 0.229532, KL Whole: 0.297530
Seed 42, Removed Class 8 | KL Forget: 0.831611, KL Retain: 0.189247, KL Whole: 0.253483
Seed 42, Removed Class 9 | KL Forget: 0.914352, KL Retain: 0.219717, KL Whole: 0.289181
Seed 602, Removed Class 0 | KL Forget: 0.793193, KL Retain: 0.180809, KL Whole: 0.242048
Seed 602, Removed Class 1 | KL 

# KL per Seed

In [1]:
import numpy as np
from lib.utils import model_results_from_npz
import csv

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 10
EPOCHS = 50
REMOVED_CLASSES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"
EPS = 1e-12

# --------------------
# Prepare CSV
# --------------------
csv_path = "../analytics/CIFAR10/kl/kl_summary_per_seed.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Seed", "KL_Forget_avg", "KL_Retain_avg", "KL_Whole_avg"])

    # --------------------
    # Loop over seeds
    # --------------------
    for seed in SEEDS:
        kl_forget_list, kl_retain_list, kl_whole_list = [], [], []

        for removed_class in REMOVED_CLASSES:
            # Load models
            retrain_path = f"{DIR_RETRAIN}/cifar_resnet_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # KL on forget class
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            conf_retrain_forget /= (conf_retrain_forget.sum(axis=1, keepdims=True) + EPS)
            conf_redis_forget   /= (conf_redis_forget.sum(axis=1, keepdims=True) + EPS)
            kl_forget_list.append(np.sum(conf_retrain_forget * np.log((conf_retrain_forget + EPS) / (conf_redis_forget + EPS)), axis=1).mean())

            # KL on retain classes
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c]   for c in retain_classes])
            conf_retrain_retain /= (conf_retrain_retain.sum(axis=1, keepdims=True) + EPS)
            conf_redis_retain   /= (conf_redis_retain.sum(axis=1, keepdims=True) + EPS)
            kl_retain_list.append(np.sum(conf_retrain_retain * np.log((conf_retrain_retain + EPS) / (conf_redis_retain + EPS)), axis=1).mean())

            # KL on whole dataset
            conf_retrain_all = np.vstack([model_retrain.confidences[c] for c in range(NUM_CLASSES)])
            conf_redis_all   = np.vstack([model_redis.confidences[c]   for c in range(NUM_CLASSES)])
            conf_retrain_all /= (conf_retrain_all.sum(axis=1, keepdims=True) + EPS)
            conf_redis_all   /= (conf_redis_all.sum(axis=1, keepdims=True) + EPS)
            kl_whole_list.append(np.sum(conf_retrain_all * np.log((conf_retrain_all + EPS) / (conf_redis_all + EPS)), axis=1).mean())

        # Average over all removed classes for this seed
        kl_forget_avg = np.mean(kl_forget_list)
        kl_retain_avg = np.mean(kl_retain_list)
        kl_whole_avg  = np.mean(kl_whole_list)

        writer.writerow([seed, kl_forget_avg, kl_retain_avg, kl_whole_avg])
        print(f"Seed {seed} | KL Forget Avg: {kl_forget_avg:.6f}, KL Retain Avg: {kl_retain_avg:.6f}, KL Whole Avg: {kl_whole_avg:.6f}")

print(f"CSV saved to {csv_path}")


Seed 42 | KL Forget Avg: 0.858001, KL Retain Avg: 0.192597, KL Whole Avg: 0.259137
Seed 602 | KL Forget Avg: 0.875728, KL Retain Avg: 0.199353, KL Whole Avg: 0.266990
Seed 311 | KL Forget Avg: 0.827141, KL Retain Avg: 0.176151, KL Whole Avg: 0.241250
Seed 637 | KL Forget Avg: 0.831394, KL Retain Avg: 0.185333, KL Whole Avg: 0.249939
Seed 800 | KL Forget Avg: 0.872069, KL Retain Avg: 0.203452, KL Whole Avg: 0.270313
Seed 543 | KL Forget Avg: 0.885885, KL Retain Avg: 0.194951, KL Whole Avg: 0.264044
Seed 969 | KL Forget Avg: 0.952484, KL Retain Avg: 0.201326, KL Whole Avg: 0.276442
Seed 122 | KL Forget Avg: 0.848031, KL Retain Avg: 0.184888, KL Whole Avg: 0.251202
Seed 336 | KL Forget Avg: 0.828714, KL Retain Avg: 0.180908, KL Whole Avg: 0.245688
Seed 93 | KL Forget Avg: 0.831520, KL Retain Avg: 0.177590, KL Whole Avg: 0.242983
CSV saved to ../analytics/CIFAR10/kl/kl_summary_per_seed.csv


# KL Three-way

In [2]:
# %%
import os
import numpy as np
import csv
from lib.utils import model_results_from_npz

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 10
EPOCHS = 50
REMOVED_CLASSES = list(range(NUM_CLASSES))
DIR_ORIG   = "../results/cifar10/"
DIR_RETRAIN = "../results/cifar10/"
DIR_REDIS   = "../results/redis_cifar10/"
EPS = 1e-12

# --------------------
# Output CSV
# --------------------
csv_path = "../analytics/CIFAR10/kl/kl_original_retrain_redis.csv"
os.makedirs(os.path.dirname(csv_path), exist_ok=True)

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Seed", "Removed_Class",
        "KL_Orig_Redis_Forget", "KL_Orig_Redis_Retain",
        "KL_Orig_Retrain_Forget", "KL_Orig_Retrain_Retain",
        "KL_Redis_Retrain_Forget", "KL_Redis_Retrain_Retain"
    ])

    # --------------------
    # Loop over seeds & removed classes
    # --------------------
    for seed in SEEDS:
        print(f"\n=== Seed {seed} ===")
        for removed_class in REMOVED_CLASSES:
            # Paths
            orig_path    = f"{DIR_ORIG}/cifar_resnet_s{seed}_e{EPOCHS}.npz"
            retrain_path = f"{DIR_RETRAIN}/cifar_resnet_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            if not (os.path.exists(orig_path) and os.path.exists(retrain_path) and os.path.exists(redis_path)):
                print(f"  Skipping class {removed_class} — missing file.")
                continue

            # Load models
            model_orig    = model_results_from_npz(orig_path, NUM_CLASSES)
            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # --------------------
            # Helper function
            # --------------------
            def kl_between(conf_a, conf_b):
                conf_a = conf_a / (conf_a.sum(axis=1, keepdims=True) + EPS)
                conf_b = conf_b / (conf_b.sum(axis=1, keepdims=True) + EPS)
                return np.sum(conf_a * np.log((conf_a + EPS) / (conf_b + EPS)), axis=1).mean()

            # --------------------
            # Split into forget / retain subsets
            # --------------------
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_orig_forget    = model_orig.confidences[removed_class]
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            conf_orig_retain    = np.vstack([model_orig.confidences[c] for c in retain_classes])
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c] for c in retain_classes])

            # --------------------
            # Compute all pairwise KL
            # --------------------
            kl_orig_redis_forget   = kl_between(conf_orig_forget,   conf_redis_forget)
            kl_orig_redis_retain   = kl_between(conf_orig_retain,   conf_redis_retain)
            kl_orig_retrain_forget = kl_between(conf_orig_forget,   conf_retrain_forget)
            kl_orig_retrain_retain = kl_between(conf_orig_retain,   conf_retrain_retain)
            kl_redis_retrain_forget = kl_between(conf_retrain_forget, conf_redis_forget)
            kl_redis_retrain_retain = kl_between(conf_retrain_retain, conf_redis_retain)

            # --------------------
            # Write to CSV
            # --------------------
            writer.writerow([
                seed, removed_class,
                kl_orig_redis_forget, kl_orig_redis_retain,
                kl_orig_retrain_forget, kl_orig_retrain_retain,
                kl_redis_retrain_forget, kl_redis_retrain_retain
            ])

            print(
                f"  Class {removed_class}: "
                f"O-Rd(F)={kl_orig_redis_forget:.3f}, O-Rd(R)={kl_orig_redis_retain:.3f} | "
                f"O-Rt(F)={kl_orig_retrain_forget:.3f}, O-Rt(R)={kl_orig_retrain_retain:.3f} | "
                f"Rd-Rt(F)={kl_redis_retrain_forget:.3f}, Rd-Rt(R)={kl_redis_retrain_retain:.3f}"
            )

print(f"\nCSV saved to {csv_path}")



=== Seed 42 ===
  Class 0: O-Rd(F)=25.440, O-Rd(R)=0.251 | O-Rt(F)=10.520, O-Rt(R)=0.287 | Rd-Rt(F)=0.788, Rd-Rt(R)=0.186
  Class 1: O-Rd(F)=26.535, O-Rd(R)=0.126 | O-Rt(F)=15.116, O-Rt(R)=0.276 | Rd-Rt(F)=0.848, Rd-Rt(R)=0.186
  Class 2: O-Rd(F)=24.257, O-Rd(R)=0.278 | O-Rt(F)=10.505, O-Rt(R)=0.295 | Rd-Rt(F)=0.843, Rd-Rt(R)=0.170
  Class 3: O-Rd(F)=23.205, O-Rd(R)=0.527 | O-Rt(F)=11.226, O-Rt(R)=0.392 | Rd-Rt(F)=0.730, Rd-Rt(R)=0.175
  Class 4: O-Rd(F)=25.630, O-Rd(R)=0.246 | O-Rt(F)=11.985, O-Rt(R)=0.303 | Rd-Rt(F)=0.863, Rd-Rt(R)=0.193
  Class 5: O-Rd(F)=23.744, O-Rd(R)=0.339 | O-Rt(F)=12.401, O-Rt(R)=0.323 | Rd-Rt(F)=0.832, Rd-Rt(R)=0.159
  Class 6: O-Rd(F)=25.927, O-Rd(R)=0.165 | O-Rt(F)=12.369, O-Rt(R)=0.287 | Rd-Rt(F)=1.020, Rd-Rt(R)=0.219
  Class 7: O-Rd(F)=25.964, O-Rd(R)=0.166 | O-Rt(F)=11.263, O-Rt(R)=0.275 | Rd-Rt(F)=0.910, Rd-Rt(R)=0.230
  Class 8: O-Rd(F)=26.345, O-Rd(R)=0.144 | O-Rt(F)=12.413, O-Rt(R)=0.270 | Rd-Rt(F)=0.832, Rd-Rt(R)=0.189
  Class 9: O-Rd(F)=26.046, O-